# Neural Networks - Learning XNOR Gates

XNOR is a nonlinear function. Can a 1-hidden layer neural network learn it? Because of the non-linearity we cannot linearly separate the classification.

In [1]:
import numpy as np
import tensorflow as tf
np.set_printoptions(precision=4, suppress=True)

train_x = np.array(
    [[0,0],[0,1],[1,0],[1,1]],
    dtype=np.float32)
train_y = np.array(
    [[1],[0],[0],[1]],
    dtype=np.float32
)

print train_x
print
print train_y

[[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]]

[[ 1.]
 [ 0.]
 [ 0.]
 [ 1.]]


In [2]:
def weight_and_bias(shape):
    W = tf.Variable(tf.random_normal(shape))
    b = tf.Variable(tf.random_normal([shape[1]]))
    return W, b


x = tf.placeholder(tf.float32, [None, 2])

N = 9

# Input -> Hidden Layer #1
W1 = tf.Variable(tf.random_normal([2,N]))
b1 = tf.Variable(tf.random_normal([N]))
a1 = tf.nn.sigmoid(tf.matmul(x,W1) + b1)

# Hidden Layer #1 -> Output Layer
W2 = tf.Variable(tf.random_normal([N,1]))
b2 = tf.Variable(tf.random_normal([1]))
a2 = tf.nn.sigmoid(tf.matmul(a1,W2) + b2)

y = a2
y_ = tf.placeholder(tf.float32, [None, 1])


print 'x shape:   ', train_x.shape
print 'a1 shape:  ', a1.get_shape()
print 'a2 shape:  ', y.get_shape()
print 'Out shape: ', y.get_shape()
print 'Data shape:', y_.get_shape()

x shape:    (4, 2)
a1 shape:   (?, 9)
a2 shape:   (?, 1)
Out shape:  (?, 1)
Data shape: (?, 1)


In [3]:
#error = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
#error = tf.reduce_sum(tf.abs(y_ - y))
error = tf.nn.l2_loss(y_ - y)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(error)

is_correct = tf.equal(tf.round(y),tf.round(y_))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [4]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

MAX_ITER = 10000
iter_count = 0
current_accuracy = 0
                  
print '= Logging ='
while (current_accuracy != 1.0 or iter_count < 2000) and (iter_count < MAX_ITER):
    sess.run(train_step, feed_dict={x:train_x, y_:train_y})
    current_accuracy = sess.run(accuracy, feed_dict={x:train_x, y_:train_y})
    current_error = sess.run(error, feed_dict={x:train_x, y_:train_y})
                                     
    # logging
    if (iter_count % 1000) == 0:
        print 'Iteration {0} - Accuracy={1}, Error={2}'.format(
                iter_count, current_accuracy, current_error)
        
    iter_count += 1
                               
    
print '\n= Results ='
print 'Iterations={0}, Accuracy={1}, Error={2}'.format(
    iter_count, current_accuracy, current_error) 



= Logging =
Iteration 0 - Accuracy=0.5, Error=0.525525093079
Iteration 1000 - Accuracy=1.0, Error=0.00813986267895

= Results =
Iterations=2000, Accuracy=1.0, Error=0.00281490245834


In [5]:
out = sess.run(y, feed_dict={x:train_x})

print 'W2\n', W2.eval(session=sess)
print
print 'b2\n', b2.eval(session=sess)
print
print 'y_hat =\n', out
print
print 'y = \n', train_y

W2
[[ 2.9396]
 [ 0.3349]
 [ 6.3478]
 [-2.6549]
 [ 4.1968]
 [-7.0099]
 [ 0.332 ]
 [-2.628 ]
 [-0.1161]]

b2
[-1.2442]

y_hat =
[[ 0.968 ]
 [ 0.0366]
 [ 0.0393]
 [ 0.9585]]

y = 
[[ 1.]
 [ 0.]
 [ 0.]
 [ 1.]]


In [6]:
# print the hidden layer

A1 = a1.eval(session=sess, feed_dict={x:train_x})
print A1

[[ 0.1106  0.4977  0.8457  0.7198  0.8625  0.1067  0.2995  0.8386  0.5412]
 [ 0.4876  0.6559  0.0379  0.2626  0.9968  0.8883  0.6545  0.5107  0.4833]
 [ 0.227   0.5253  0.0365  0.7084  0.1776  0.0006  0.2367  0.7172  0.6935]
 [ 0.6921  0.6804  0.0003  0.2518  0.9155  0.0406  0.5789  0.3375  0.6421]]
